# Jet pt and mass "inverting" regression with omnifold

In [1]:
from __future__ import absolute_import, division, print_function
import numpy as np
import uproot
import aiohttp
from matplotlib import pyplot as plt
import awkward as ak
from keras.layers import Dense, Input
from keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

import omnifold 
from omnifold import DataLoader, MultiFold, MLP, SetStyle, HistRoutine
import os
import tensorflow as tf
import coffea
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import matplotlib as mpl
#####
#Delta R matched

2025-02-04 20:13:03.438765: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-04 20:13:03.441802: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-02-04 20:13:03.451534: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-04 20:13:03.467133: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-04 20:13:03.471473: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-04 20:13:03.481951: I tensorflow/core/platform/cpu_feature_gu

Horovod not found, will continue with single only GPUs.


/opt/conda/lib/python3.11/site-packages/coffea/nanoevents/schemas/fcc.py:5: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import vector


In [2]:

import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
mpl.rcParams["text.usetex"]=False 

In [3]:
coffea.__version__

'2024.10.0'

### Check to see if a GPU is present for acceleration

### Get leading generator-level ("truth" or "gen") jet and match to the nearest reconstructed-level ("reco") jet

In [4]:
#coffea init
cofevents = NanoEventsFactory.from_root( {'QCD_Pt-15to7000_TuneCP5_Flat2018_13TeV_pythia8_NANOAODSIM_7F142AC6-39F2-2F4F-9F97-C425A1C99A35.root':'Events'}, schemaclass=NanoAODSchema).events()
####

# Filter 0s 
cevents = cofevents[ak.num(cofevents.FatJet)>0]
cevents = cevents[ak.num(cevents.GenJetAK8)>0]
fj = cevents.FatJet
gj = cevents.GenJetAK8
#fsubjets = cevents.SubJetsAK8
gsubjets = cevents.SubGenJetAK8


#this should give fatjets closest to genjets, and since genjets are already stripped of zeroes, this should just line up (pfft just kidding)
nearest_fj = gj.nearest(fj)

gj0 = gj[:,0]          # Leading generator-level (gen) jet
fj0 = nearest_fj[:,0]  # Reco jet that matches leading gen jet

/opt/conda/lib/python3.11/site-packages/coffea/nanoevents/schemas/nanoaod.py:264: RuntimeWarning: Missing cross-reference index for LowPtElectron_electronIdx => Electron
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/coffea/nanoevents/schemas/nanoaod.py:264: RuntimeWarning: Missing cross-reference index for LowPtElectron_photonIdx => Photon
  warnings.warn(


In [74]:
ak.num(gj, axis=1) > 0

dask.awkward<greater, npartitions=1>

In [75]:
sel = ak.num(gj, axis=1) > 0

In [79]:
ak.sum(sel).compute()

300792

In [81]:
len(gj.compute())

300792

In [84]:
gj0.compute()

<PtEtaPhiMCollectionArray [{eta: -0.574, mass: 61.2, ...}, ...] type='28438...'>

In [37]:
def get_groomed_jet( jet, subjets , verbose = False):
    '''
    Find the subjets that correspond to the given jet using delta R matching. 
    This is suboptimal, but it's hard to fix upstream. 
    '''
    n_subjets = ak.num( subjets, axis=1 )
    n_jet = ak.num(jet, axis=0)
    sel = (n_jet >= 1) & (n_subjets >= 2)
    combs = ak.cartesian( (jet[sel], subjets[sel]), axis=1 )
    dr_jet_subjets = combs['0'].delta_r(combs['1'])
    sel = dr_jet_subjets < 0.8
    total = combs[sel]['1'].sum(axis=1)
    return total, sel

In [65]:
gsubjets[0].compute()

<PtEtaPhiMCollectionArray [{eta: -0.562, mass: 3.36, ...}, ...] type='4 * P...'>

In [38]:
gj0_groomed, gj0_groomed_select = get_groomed_jet( gj0, gsubjets )

n_subjets_ge_2  dask.awkward<greater-equal, npartitions=1>


IndexError: cannot slice IndexedOptionArray (of length 284388) with array([     0,      1,      2, ..., 300789, 300790, 300791]): index 284388 is out of bounds for axis 0 with size 284388

### Compute $\Delta R$ between leading gen jet and matched reco jet. Require to be $< 0.2$

In [7]:
# Calculate delta r
gj_deltar_fj = gj0.delta_r(fj0)
# Find events where the delta r is < 0.2
pass_dr = gj_deltar_fj < 0.2
gj0 = gj0[pass_dr]
fj0 = fj0[pass_dr]
# Discard events with low pt
pass_pt = fj0.pt > 200.
gj0 = gj0[pass_pt]
fj0 = fj0[pass_pt]
gj0_groomed = gj0_groomed[pass_pt]

### Divide into even and odd events to test and train the neural network

In [8]:
#pick lowest size
size_odd = len(gj0.pt[1::2])
size_even = len(gj0.pt[0::2])
size_all = size_even if size_odd > size_even else size_odd

In [9]:
even_gj0 = gj0[0::2][0:size_all]
odd_gj0 = gj0[1::2][0:size_all]

even_gj0_groomed = gj0_groomed[0::2][0:size_all]
odd_gj0_groomed = gj0_groomed[1::2][0:size_all]

even_fj0 = fj0[0::2][0:size_all]
odd_fj0 = fj0[1::2][0:size_all]

### Arrange the jet pt and mass into the format omnifold expects

General plan is to add jet composition variables to this array and see if the resolution of the regression improves

In [10]:

odd_gen = np.column_stack((
    ak.to_numpy( odd_gj0.mass.compute() ),
    ak.to_numpy( odd_gj0.pt.compute() ),
    ak.to_numpy( odd_gj0_groomed.mass.compute() )
))

odd_reco = np.column_stack((
    ak.to_numpy( odd_fj0.mass.compute() ),
    ak.to_numpy( odd_fj0.pt.compute() ),
    ak.to_numpy( odd_fj0.msoftdrop.compute() )
))

even_gen = np.column_stack([
    ak.to_numpy(even_gj0.mass.compute() ),
    ak.to_numpy(even_gj0.pt.compute() ),
    ak.to_numpy( even_gj0_groomed.mass.compute() )
])

even_reco = np.column_stack([
    ak.to_numpy(even_fj0.mass.compute() ) ,
    ak.to_numpy(even_fj0.pt.compute() ),
    ak.to_numpy(even_fj0.msoftdrop.compute() )
])

theta0 = np.stack((odd_gen, odd_reco), axis=1)

print("CLEAR")

CLEAR


In [ ]:
theta0.shape

In [ ]:
print(odd_gen)
print(even_gen)
print(theta0)

In [ ]:
_,_,_=plt.hist(odd_gen[:,0],bins=np.linspace(0,1000,200),color='blue',alpha=0.5,label="Odd, gen")
_,_,_=plt.hist(odd_reco[:,0],bins=np.linspace(0,1000,200),histtype="step",color='black',ls=':',label="Odd, reco")
_,_,_=plt.hist(even_gen[:,0],bins=np.linspace(0,1000,200),color='orange',alpha=0.5,label="Even, gen")
_,_,_=plt.hist(even_reco[:,0],bins=np.linspace(0,1000,200),histtype="step",color='black',label="Even, reco")
plt.xlabel("Ungroomed jet mass $m$ (GeV)")
plt.ylabel("Number of events")
plt.legend(frameon=False)



In [ ]:
_,_,_=plt.hist(odd_gen[:,1],bins=np.linspace(-3,1000,200),color='blue',alpha=0.5,label="Odd, gen")
_,_,_=plt.hist(odd_reco[:,1],bins=np.linspace(-3,1000,200),histtype="step",color='black',ls=':',label="Odd, reco")
_,_,_=plt.hist(even_gen[:, 1],bins=np.linspace(-3,1000,200),color='orange',alpha=0.5,label="Even, gen")
_,_,_=plt.hist(even_reco[:, 1],bins=np.linspace(-3,1000,200),histtype="step",color='black',label="Even, reco")
plt.xlabel("Transverse Momentum $p_{T}$ (GeV)")
plt.ylabel("Number of events")
plt.legend(frameon=False)


In [ ]:
_,_,_=plt.hist(odd_gen[:,2],bins=np.linspace(0,1000,200),color='blue',alpha=0.5,label="Odd, gen")
_,_,_=plt.hist(odd_reco[:,2],bins=np.linspace(0,1000,200),histtype="step",color='black',ls=':',label="Odd, reco")
_,_,_=plt.hist(even_gen[:,2],bins=np.linspace(0,1000,200),color='orange',alpha=0.5,label="Even, gen")
_,_,_=plt.hist(even_reco[:,2],bins=np.linspace(0,1000,200),histtype="step",color='black',label="Even, reco")
plt.xlabel("Groomed jet mass $m$ (GeV)")
plt.ylabel("Number of events")
plt.legend(frameon=False)

In [ ]:
_,_,_=plt.hist(gj0.eta.compute(),bins=np.linspace(-3,3,200),color='blue',alpha=0.5,label="gen, eta")
_,_,_=plt.hist(gj0.phi.compute(),bins=np.linspace(-3,3,200),histtype="step",color='black',ls=':',label="gen, phi")
_,_,_=plt.hist(fj0.eta.compute(),bins=np.linspace(-3,3,200),color='orange',alpha=0.5,label="reco, eta")
_,_,_=plt.hist(fj0.phi.compute(),bins=np.linspace(-3,3,200),histtype="step",color='green',ls=':',label="reco, phi")
#_,_,_=plt.hist(jet_genmass[:],bins=np.linspace(-3,1000,200),color='orange',alpha=0.5,label="Data, true")
#_,_,_=plt.hist(jet_genpt[:],bins=np.linspace(-3,1000,200),histtype="step",color='black',label="Data, reco")
plt.xlabel("parts")
plt.ylabel("events")
plt.legend(frameon=False)

In [ ]:
#Data, true
plt.figure(figsize=(10, 8))
plt.hist2d(even_gen[:,0], even_reco[:,0], 
           bins=(np.linspace(0,1000,101), np.linspace(0,1000,101)), 
           cmap='Oranges', alpha=0.5, label="Data, true",
           norm=mpl.colors.LogNorm())
plt.colorbar()
plt.xlabel("Generated mass (GeV)")
plt.ylabel("Reconstructed mass (GeV)")
plt.title("Even events ")
plt.show()

In [ ]:
#Data, true
plt.figure(figsize=(10, 8))
plt.hist2d(even_gen[:,1], even_reco[:,1], 
           bins=(np.linspace(200,1000,101), np.linspace(200,1000,101)), 
           cmap='Oranges', alpha=0.5, label="Data, true",
           norm=mpl.colors.LogNorm())
plt.colorbar()
plt.xlabel("Generated transverse momentum (GeV)")
plt.ylabel("Reconstructed transverse momentum (GeV)")
plt.title("Even events ")
plt.show()

In [ ]:
#Data, true
plt.figure(figsize=(10, 8))
plt.hist2d(even_gen[:,2], even_reco[:,2], 
           bins=(np.linspace(0,1000,101), np.linspace(0,1000,101)), 
           cmap='Oranges', alpha=0.5, label="Data, true",
           norm=mpl.colors.LogNorm())
plt.colorbar()
plt.xlabel("Generated groomed mass (GeV)")
plt.ylabel("Reconstructed groomed mass (GeV)")
plt.title("Even events ")
plt.show()

In [ ]:
#Data, true
plt.figure(figsize=(10, 8))
plt.hist2d(even_reco[:, 0], even_reco[:, 1], bins=(np.linspace(0,100,101), np.linspace(0,1000,101)), cmap='Oranges', alpha=0.5, label="Data, true")
plt.colorbar()
plt.xlabel("Jet mass $m$ (GeV)")
plt.ylabel("Jet transverse momentum $p_T$ (GeV)")
plt.title("Even events ")
plt.show()

In [ ]:
data = DataLoader(reco = even_reco,normalize=True)
mc = DataLoader(reco = odd_reco,gen = odd_gen,normalize=True)

In [ ]:
SetStyle()
data_dict = {
    'even_gen': even_gen[:,0],
    'even_reco': even_reco[:,0],
    'odd_gen': odd_gen[:,0],
    'odd_reco': odd_reco[:,0],
}
HistRoutine(data_dict,'First Feature [a.u]', reference_name = 'even_gen',plot_ratio=False)

In [ ]:
ndim = 3
model1 = MLP(ndim)
model2 = MLP(ndim)

In [ ]:
print(model1.summary())

In [ ]:
omnifold_instance = MultiFold(
    "PtMass",
    model1,
    model2,
    data,
    mc,
    batch_size = 1024,
    niter = 5,  #Number of Iterations                                                                                                                                                                                                  
    epochs=100,     
    weights_folder = 'weights',
    verbose = True,
)

In [ ]:
omnifold_instance.Preprocessing()
omnifold_instance.Unfold()

In [ ]:
unfolded_weights  = omnifold_instance.reweight(even_gen,omnifold_instance.model2,batch_size=1000)   

`data` is the even events, `mc` is the odd events

In [ ]:
print(unfolded_weights)

In [ ]:
data_dict_mass = {
    'even_gen': even_gen[:,0],
    'even_reco': even_reco[:,0],
    'odd_gen': odd_gen[:,0],
    'odd_reco': odd_reco[:,0],
    'unfolded': even_gen[:,0]
}


weight_dict_mass = {
    'even_gen': data.weight,
    'even_reco': data.weight,
    'odd_gen': mc.weight,
    'odd_reco': mc.weight,
    'unfolded': unfolded_weights,
    }

fig,_ = HistRoutine(data_dict_mass,'Ungroomed Jet Mass ($m_u$) [GeV]',
                    reference_name = 'even_gen',
                    weights = weight_dict_mass,
                    plot_ratio = False
                    )

In [ ]:
data_dict_pt = {
    'even_gen': even_gen[:,1],
    'even_reco': even_reco[:,1],
    'odd_gen': odd_gen[:,1],
    'odd_reco': odd_reco[:,1],
    'unfolded': even_gen[:,1]
}


weight_dict_pt = {
    'even_gen': data.weight,
    'even_reco': data.weight,
    'odd_gen': mc.weight,
    'odd_reco': mc.weight,
    'unfolded': unfolded_weights,
    }

fig,_ = HistRoutine(data_dict_pt,'Jet Transverse Momentum ($p_T$) [GeV]',
                    reference_name = 'even_gen',
                    weights = weight_dict_pt,
                    plot_ratio = False
                    )